In [1]:
%load_ext autoreload
%autoreload 2

### Import

In [2]:
import sys
sys.path.append('..')
from tqdm import tqdm
import pandas as pd

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import pickle

import gensim.downloader
from gensim.models import FastText


from sklearn.metrics import classification_report

from constants import CATEGORIES

In [3]:
# CATEGORIES = ['overall_toxic']

#### GPU Usage

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
print(torch.cuda.get_device_name(0))

Using device: cpu


### Import Data

In [5]:
df_train = pd.read_parquet('data/df_train_preprocessed.parquet')
df_val = pd.read_parquet('data/df_val_preprocessed.parquet')
df_test = pd.read_parquet('data/df_test_preprocessed.parquet')

In [7]:
preprocess_types = ['baseline',
                    'word_tokenize_simple_normalization',
                    'word_tokenize_full_normalization',
                    'bpe_tokenize_no_dup_no_punc_normalization',
                    'bpe_tokenize_simple_dup_normalization',
                    ]

#### Utils

In [8]:
def itemget(model_vector, nb_feature: int, item:str):
    try:
        return model_vector[item]
    except:
        return np.zeros(nb_feature)

In [9]:
def mean(list_vectors, nb_feature: int) -> np.ndarray:
    if len(list_vectors) == 0:
        return np.zeros(nb_feature)
    return np.mean(list_vectors, axis=0)

In [10]:
def vectorize_and_split(model_vector, nb_feature,preprocess_type: str) -> tuple:
    X_train = df_train[f"comment_text_{preprocess_type}"].apply(lambda x: mean([itemget(model_vector,nb_feature,word) for word in x.split()], nb_feature))
    X_valid = df_val[f"comment_text_{preprocess_type}"].apply(lambda x: mean([itemget(model_vector,nb_feature,word)  for word in x.split()], nb_feature))
    X_test = df_test[f"comment_text_{preprocess_type}"].apply(lambda x: mean([itemget(model_vector,nb_feature,word)  for word in x.split()], nb_feature))
    return (np.vstack(X_train.to_numpy()), np.vstack(X_valid.to_numpy()), np.vstack(X_test.to_numpy()), model_vector)

In [11]:
def vectorize_split_and_pickle(model, vector, filename):
    Xs_pre_trained = {preprocess_type: vectorize_and_split(model, vector, preprocess_type) for preprocess_type in preprocess_types}
    with open(f'data/{filename}.pickle', 'wb') as file:
        pickle.dump(Xs_pre_trained, file, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
def input_generator(embeddings,normalization_type):
    X_train, X_val, X_test, _ = embeddings[normalization_type]
    return X_train, X_val, X_test

In [13]:
def calculate_class_weights(df, categories, num_labels):
    class_counts = np.sum(df[categories].values, axis=0)
    class_weights = len(df) / class_counts
    class_weights /= (num_labels + 1)
    return class_weights

In [14]:
dim = 200

In [15]:
class TextDataset(Dataset):
    def __init__(self, vectors, labels, model, vector_size):
        self.vectors = vectors
        self.labels = labels
        self.model = model
        self.vector_size = vector_size

    def __len__(self):
        return self.vectors.shape[0]

    def __getitem__(self, idx):
            label = self.labels[idx]
            embedding = self.vectors[idx]
            embedding = torch.tensor(embedding, dtype=torch.float32)
            label_tensor = torch.tensor(label, dtype=torch.float32)
            return embedding, label_tensor


In [16]:
def prepare_data(vectors, labels, batch_size=1024, vector_size=200, model=None):
    dataset = TextDataset(vectors, labels, model, vector_size)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    input_dim = vector_size
    return loader, input_dim

In [17]:
def evaluate_model(model, test_loader, device, categories):
    model.eval()

    y_pred = []
    y_true = []

    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            y_pred.extend(outputs.cpu().numpy())
            y_true.extend(targets.cpu().numpy())

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred) > 0.5

    y_true = y_true.numpy()
    y_pred = y_pred.numpy()

    overall_non_toxic = np.zeros(y_pred.shape[0])
    overall_non_toxic[y_pred.sum(axis=1) == 0] = 1
    y_pred = np.hstack([y_pred, overall_non_toxic.reshape(-1,1)])

    overall_non_toxic = np.zeros(y_true.shape[0])
    overall_non_toxic[y_true.sum(axis=1) == 0] = 1
    y_true = np.hstack([y_true, overall_non_toxic.reshape(-1,1)])

    print(classification_report(y_true, y_pred, target_names=categories+['overall_non_toxic']))

### Embeddings

#### Glove

In [19]:
print('\n'.join(gensim.downloader.info()['models'].keys()))
glove_vectors = gensim.downloader.load(f'glove-twitter-{dim}')
vectorize_split_and_pickle(glove_vectors, dim, f'glove-twitter-{dim}')

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


#### FastText

In [20]:
def fasttext_model_generator(df_train: pd.DataFrame, normalization_type: str, vector_size = 200, window = 5, min_count = 1, workers = 4):
    tokenized_texts = [text.split() for text in df_train[f'comment_text_{normalization_type}']]
    model = FastText(sentences=tokenized_texts, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    return model

fasttext_vectors = fasttext_model_generator(df_train, 'baseline', dim).wv
vectorize_split_and_pickle(fasttext_vectors, dim, f'fasttext_{dim}d')

In [24]:
with open('data/glove-twitter-200.pickle', 'rb') as file:
    Xs_w2v_pre_trained = pickle.load(file)

In [25]:
with open('data/fasttext_200d.pickle', 'rb') as file:
    Xs_fasttext_pre_trained= pickle.load(file)

In [26]:
embeddings = [Xs_w2v_pre_trained, Xs_fasttext_pre_trained]

#### Creation du modele

In [27]:
class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.fc_out(x)
        x = torch.sigmoid(x)
        return x

In [28]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

#### Initialisation du modèle

In [29]:
model_parameters = {
    'process_types': preprocess_types,
    'embeddings': embeddings,
    'hidden_dim': [32,64,128],
}

batch_size = 1024
epochs = 50

In [30]:
for proceprocess_types in model_parameters['process_types']:
    for embedding in model_parameters['embeddings']:
        for hidden_dim in model_parameters['hidden_dim']:
            if embedding is Xs_w2v_pre_trained:
                print(f'Processing {proceprocess_types} with glove-twitter-{dim} and hidden dimmension {hidden_dim}')
            else:
                print(f'Processing {proceprocess_types} with fasttext-{dim} and hidden dim {hidden_dim}')
            X_train, X_val, X_test = input_generator(embedding, proceprocess_types)
            y_train = df_train[CATEGORIES].values
            y_val = df_val[CATEGORIES].values
            y_test = df_test[CATEGORIES].values

            train_loader, input_dim = prepare_data(X_train, y_train, model=embedding, vector_size=dim)
            val_loader, _ = prepare_data(X_val, y_val, model=embedding, vector_size=dim)
            test_loader, _ = prepare_data(X_test, y_test, model=embedding, vector_size=dim)

            model = FNN(input_dim, hidden_dim, len(CATEGORIES)).to(device)
            criterion = nn.BCELoss(weight=torch.tensor(calculate_class_weights(df_train, CATEGORIES, len(CATEGORIES) + 1)).to(device))
            optimizer = optim.Adam(model.parameters(), lr=0.001)

            train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=epochs)
            evaluate_model(model, test_loader, device, CATEGORIES)
            print('\n\n')

Processing baseline with glove-twitter-200 and hidden dimmension 32


/Users/camilziane/.pyenv/versions/3.11.6/envs/nlp_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 32/32 [00:01<00:00, 27.92it/s]


Epoch 1/50, Train Loss: 5.2764, Val Loss: 2.8585


100%|██████████| 32/32 [00:01<00:00, 19.26it/s]


Epoch 2/50, Train Loss: 1.7962, Val Loss: 1.2405


...

Epoch 49/50, Train Loss: 0.2250, Val Loss: 0.3191


100%|██████████| 32/32 [00:01<00:00, 30.59it/s]


Epoch 50/50, Train Loss: 0.2245, Val Loss: 0.3135


100%|██████████| 63/63 [00:01<00:00, 35.61it/s]


                   precision    recall  f1-score   support

            toxic       0.58      0.59      0.58      6090
     severe_toxic       0.31      0.34      0.33       367
          obscene       0.67      0.56      0.61      3691
           threat       0.56      0.21      0.30       211
           insult       0.60      0.50      0.54      3427
    identity_hate       0.39      0.20      0.27       712
overall_non_toxic       0.96      0.96      0.96     57735

        micro avg       0.89      0.87      0.88     72233
        macro avg       0.58      0.48      0.51     72233
     weighted avg       0.88      0.87      0.88     72233
      samples avg       0.91      0.91      0.91     72233




